In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
os.getcwd()

In [ ]:
df=pd.read_csv("ozan_p_pApply_intern_challenge_03_20_min.csv", encoding='utf8')

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df.search_date_pacific=pd.to_datetime(df.search_date_pacific)
df.rename(columns={'apply':'Apply'}, inplace=True)

In [ ]:
df.shape

In [ ]:
min(df.search_date_pacific)

In [ ]:
df_train=df[df.search_date_pacific<='2018-01-26']

In [ ]:
df_test=df[df.search_date_pacific=='2018-01-27']

In [ ]:
df_train.shape[0]+df_test.shape[0]

In [ ]:
sum(df.Apply)/len(df.Apply)

In [ ]:
plt.hist(df.Apply)
plt.title("Apply Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")

In [ ]:
df.isnull().any()
#no missing values

In [ ]:
X_train=df_train.iloc[:,0:7]
Y_train=df_train.Apply
X_test=df_test.iloc[:,0:7]
Y_test=df_test.Apply

In [ ]:
Y_test.head()

# SVM

In [ ]:
from sklearn import svm
clf = svm.SVC()

clf.fit(X_train,Y_train)

In [ ]:
clf

# NN based

In [ ]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM,SimpleRNNCell
from keras.layers.embeddings import Embedding
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
from keras.utils import np_utils
Y_test_nn=np_utils.to_categorical(Y_test,2)
Y_train_nn=np_utils.to_categorical(Y_train,2)

In [ ]:
model=Sequential()
model.add(Dense(256,input_dim=7))
Activation('relu')
model.add(Dropout(0.25))
model.add(Dense(64,input_dim=256))
model.add(Dropout(0.25))
model.add(Dense(64,input_dim=256))
model.add(Dropout(0.25))
model.add(Dense(64,input_dim=256))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
X_train.shape

In [ ]:
model.fit(X_train,Y_train_nn,batch_size=200,epochs=5)

In [ ]:
score = model.evaluate(X_test, Y_test_nn, verbose=0)

In [ ]:
score

In [ ]:
Y_pre_nn=model.predict_classes(X_test)
len(Y_pre_nn)

In [ ]:
#evaluate model
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, Y_pre_nn)
confusion_matrix

In [ ]:
import theano
import theano.tensor as T

epsilon = 1.0e-9
def custom_objective(y_true, y_pred):
    '''Just another crossentropy'''
    y_pred = T.clip(y_pred, epsilon, 1.0 - epsilon)
    y_pred /= y_pred.sum(axis=-1, keepdims=True)
    cce = T.nnet.categorical_crossentropy(y_pred, y_true)
    return cce

In [ ]:
#costune loss
def custom_objective(y_true, y_pred):
    '''Just another crossentropy'''
    print(type(y_true))
    npy_true=np.array(y_true).astype(int)
    npy_pred=np.array(y_pred).astype(int)
    tmpminus=(npy_true)-(npy_pred)
    
    los=tmpminus[tmpminus==-1]*10+tmpminus[tmpminus==1]
    return np.sum(los)

In [ ]:
a=[True,False]
a*1
np.array(a).astype(int)

In [ ]:
model.compile(loss=custom_objective,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train,Y_train_nn,batch_size=200,epochs=5)

In [ ]:
Y_pre_nn=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, Y_pre_nn)
confusion_matrix


In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_pre_nn)
metrics.auc(fpr, tpr)

# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LogReg = LogisticRegression()
LogReg.fit(X_train, Y_train)

In [ ]:
#evaluate model
Y_pred_logit = LogReg.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, Y_pred_logit)
confusion_matrix

In [ ]:
#predict probability
probY_test=LogReg.predict_proba(X_test)
Y_pred=LogReg.predict(X_test)

In [ ]:
plt.hist(Y_test)
plt.title("Apply Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_pred_logit)
metrics.auc(fpr, tpr)
#not good!!!!!!!

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(X_train, Y_train)

In [ ]:
Y_pred_rd=clf.predict(X_test)


In [ ]:
sum(Y_pred_rd)

In [ ]:
#evaluate model
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, Y_pred_rd)
confusion_matrix

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_pred_rd)
metrics.auc(fpr, tpr)

In [ ]:
#not good also...

# Single Tree based method


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth =1000,max_features=7)
clf.fit(X_train, Y_train)

In [ ]:
Y_pre_tree=clf.predict(X_test)

In [ ]:
sum(Y_pre_tree)
#evaluate model
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, Y_pre_tree)
confusion_matrix

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_pre_tree)
metrics.auc(fpr, tpr)

# Bayesian network